https://aviris.jpl.nasa.gov/dataportal/

In [1]:
import requests
import tarfile
import re
import csv
import numpy as np
import torch
from torchvision import transforms
from IPython.display import display
from datasets import Dataset, concatenate_datasets, Features, Array3D

In [2]:
def is_header(filename):
    return filename.endswith('_img.hdr')
def is_data(filename):
    return filename.endswith('_img')
def get_size(header):
    nsamp_match = re.search(r"samples\s*=\s*(\d+)", header)
    nlines_match = re.search(r"lines\s*=\s*(\d+)", header)
    nbands_match = re.search(r"bands\s*=\s*(\d+)", header)
    nsamp = int(nsamp_match.group(1))
    nlines = int(nlines_match.group(1))
    nbands = int(nbands_match.group(1))
    return nsamp, nlines, nbands
def pt_to_pil(t):
    t = t+0.5
    t = t*255
    t = torch.clamp(t, min=-0.49, max=255.49)
    t = t.round()
    t = t.to(torch.uint8)
    return t
def visualize_hyper(x):
    z = torch.tensor(x[[20,100,180],:,:])
    z = z.to(torch.float)
    z = z - z.mean()
    z = 3 * z / z.std()
    z = torch.clamp(z,min=-3,max=3)
    z = z / 6
    display(transforms.ToPILImage()(pt_to_pil(z)))
def is_empty(x):
    z = torch.tensor(x)
    z = z.to(torch.float)
    z = z - z.mean()
    z = 3 * z / z.std()
    z = torch.clamp(z,min=-3,max=3)
    z = z / 6
    empty = False
    if (z.std()<0.3 or torch.isnan(z.std())):
        empty = True
    return empty

In [3]:
url_list = []
with open('aviris_2010_2011.csv', 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    for row in csvreader:
        url = row[0]
        url_list.append(url)

In [ ]:
for batch in range(2,40):
    try:
        start_ind = 5*batch
        end_ind = start_ind+5
        dataset_list = []
        for url in url_list[start_ind:end_ind]:
            try:
                print("downloading",url,"...")
                response = requests.head(url)
                file_size = int(response.headers['content-length'])
                assert(file_size < 2e9)
                response = requests.get(url)
                with open(url[-23:], 'wb') as f:
                    f.write(response.content)
                with tarfile.open(url[-23:], 'r:gz') as tar:
                    names = tar.getnames()
                    header = next(filter(is_header, names), None)   
                    header = tar.extractfile(header)
                    header = header.read().decode('utf-8')
                    nsamp, nlines, nbands = get_size(header)
                    shape = (nlines, nsamp, nbands)
                    dtype = np.int16
                    data = next(filter(is_data, names), None)
                    x = tar.extractfile(data).read()
                    x = np.frombuffer(x, dtype=dtype)
                    x = x.reshape(shape)
                    x = x.transpose((2,0,1))
                    C, H, W = x.shape
                    new_H = (H // 256) * 256
                    new_W = (W // 256) * 256
                    start_H = (H - new_H) // 2
                    start_W = (W - new_W) // 2
                    y = x[:, start_H:start_H + new_H, start_W:start_W + new_W]
                    tiles = []
                    for h in range(0, new_H, 256):
                        for w in range(0, new_W, 256):
                            tile = y[:, h:h+256, w:w+256]
                            if not is_empty(tile):
                                tiles.append(tile)
                    features = Features({'hyperspectral': Array3D(shape=(224, 256, 256), dtype='int16')})
                    dataset = Dataset.from_dict({"hyperspectral": tiles}, features=features)
                    dataset_list.append(dataset)
            except:
                pass
        dataset = concatenate_datasets(dataset_list)
        dataset = dataset.with_format("torch")
        dataset.push_to_hub("danjacobellis_AVIRIS_256",split=f"batch{batch}")
    except:
        pass

downloading https://popo.jpl.nasa.gov/avcl/y11_data/f110817t01p00r13.tar.gz ...
downloading https://popo.jpl.nasa.gov/avcl/y11_data/f110615t01p00r06.tar.gz ...
downloading https://popo.jpl.nasa.gov/avcl/y10_data/f100929t01p00r07.tar.gz ...
downloading https://popo.jpl.nasa.gov/avcl/y10_data/f100929t01p00r06.tar.gz ...
downloading https://popo.jpl.nasa.gov/avcl/y11_data/f110807t01p00r22.tar.gz ...


Pushing dataset shards to the dataset hub:   0%|          | 0/10 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

downloading https://popo.jpl.nasa.gov/avcl/y11_data/f110615t01p00r16.tar.gz ...
downloading https://popo.jpl.nasa.gov/avcl/y11_data/f110506t01p00r12.tar.gz ...
downloading https://popo.jpl.nasa.gov/avcl/y10_data/f100731t01p00r23.tar.gz ...
downloading https://popo.jpl.nasa.gov/avcl/y10_data/f100731t01p00r11.tar.gz ...
downloading https://popo.jpl.nasa.gov/avcl/y10_data/f100704t01p00r07.tar.gz ...


Pushing dataset shards to the dataset hub:   0%|          | 0/13 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]